## Scraping y recolección de datos de la web de Facua

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from time import sleep
import json
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd


# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

El primer paso de este proyecto es obtener para cada uno de los supermercados de la web de facua el precio a lo largo del tiempo de tres tipo de productos: aceite de giralos, aceite de oliva y leche. Para ello usaremos Beautiful soup y selenium: 

In [59]:
driver = webdriver.Chrome()
url="https://super.facua.org/"
driver.get(url)
driver.maximize_window()
sleep(1)

# Diccionario donde por super y categoría almacenaremos los links de las páginas de los productos
diccionario_url = {
    "mercadona":{},
    "carrefour":{},
    "eroski":{},
    "dia":{},
    "hipercor":{},
    "alcampo":{},
}

# Itero por supermercado
for i in range(1,7):
    sleep(1)
    driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-6.row-cols-2.row-cols-md-2.row-cols-xl-6.justify-content-center > div:nth-child({i}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
    sleep(2)

    aux_elements=driver.find_elements("css selector", "body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center")
    aux = [element.text for element in aux_elements]
    lista_productos = aux[0].split("\n")[::2]
    
    j=1
    for producto in lista_productos:
        
        # -------------Aceite de girasol, aceite de oliva, leche------------------
        driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({j}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
        sleep(2)

        #Saco los link de los productos de aceite de girasol
        html_table_page = driver.page_source
        sopa = BeautifulSoup(html_table_page, "html.parser")
        productos = sopa.findAll("div", {"class":"card-footer p-4 pt-0 border-top-0 bg-transparent"})
        lista_links = []
        for prod in productos:
            link = prod.find("a", {"class":"btn-unirme btn-verde inline-block inline-block bg-primary border-primary font-semibold rounded-full"}).get("href")
            lista_links.append(link)
        
        #Sacamos la clave correspondiente para el diccionario
        clave = list(diccionario_url.keys())[i-1]
        diccionario_url[clave][producto]=lista_links

        driver.back()
        sleep(1)
        j+=1

    driver.back()
    sleep(2)
    
driver.quit()

In [64]:
with open("../datos/diccionario_links.json", "w") as archivo:
    json.dump(diccionario_url, archivo)

Cargamos el diccionario

In [68]:
with open("../datos/diccionario_links.json", "r") as archivo:
    diccionario_url = json.load(archivo)

Creamos un dataframe para cada producto y supermercado

In [135]:
# iteramos para cada super
for clave, valor in tqdm(diccionario_url.items()):
    #Iteramos para cada producto
    for producto in lista_productos:
        links = diccionario_url[clave][producto]
        df_producto = pd.DataFrame()
        for link in links:
            # Meto un try excepy porque hay algunos links que no son de productos sino de una categoría
            try:
                url = link
                contenido = requests.get(url)
                contenido.status_code
                sopa = BeautifulSoup(contenido.content, "html.parser")

                tabla = sopa.find("table", {"class":"table table-striped table-responsive text-center"})

                enunciados = [enunciado.getText() for enunciado in tabla.findAll("th")]

                filas_tabla=[]
                for fila in tabla.findAll("tr"):
                    fila_sucia= fila.findAll("td")
                    aux=[elem.text for elem in fila_sucia]
                    filas_tabla.append(aux)
                
                df= pd.DataFrame(filas_tabla[1:], columns=enunciados)
                df["supermercado"] = clave
                df["producto"] = producto
                df["nombre_producto"] = url.split("/")[-2].replace("-"," ")
                df_producto = pd.concat([df_producto, df])
                
            except:
                pass

        df_producto.to_csv(f"../datos/df_{clave}_{producto}.csv")

100%|██████████| 6/6 [04:30<00:00, 45.09s/it]
